In [1]:
import uproot
import numpy as np
from os import listdir
from os.path import isfile, join
import os
import math

## Einfuehrung

In [ ]:
myfolder = "/nfs/freenas/tuph/e18/project/compass/analysis/tkumschier/ROOTPWA/test/TEST_FOLDER/ROOTPWA_MC_TEST0/amps/"
myfileName = "[1-,1+,0+]=[piPiSWaveAuMorganPenningtonKachaev[sigma0]=[pi-[0,0]pi+][1,0]pi-]_eventFileId-0_2.root"
myRootFile = uproot.open(myfolder + myfileName)

In [ ]:
print myRootFile.keys()
print myRootFile[myRootFile.keys()[0]]

In [ ]:
print myRootFile[myRootFile.keys()[0]].keys()
print myRootFile[myRootFile.keys()[0]]['amplitude'].show()
print myRootFile[myRootFile.keys()[0]]['amplitude'].get('_incohSubAmps').array()

In [ ]:
incohSubAmps = myRootFile[myRootFile.keys()[0]]['amplitude'].get('_incohSubAmps').array()
incohSubAmpsReal = myRootFile['[1-,0-,0+]=[f0_1500_0=[pi-[0,0]pi+][0,0]pi-].amp']['amplitude'].get('_incohSubAmps._real').array()
incohSubAmpsImag = myRootFile['[1-,0-,0+]=[f0_1500_0=[pi-[0,0]pi+][0,0]pi-].amp']['amplitude'].get('_incohSubAmps._imag').array()
#print file['[1-,0-,0+]=[f0_1500_0=[pi-[0,0]pi+][0,0]pi-].amp']['amplitude'].get('_incohSubAmps').array()

In [ ]:
incohSubAmpsComplex = np.array(incohSubAmpsReal + 1j * incohSubAmpsImag)
print sum(incohSubAmpsComplex * incohSubAmpsComplex.conjugate())/nmbEvents
print np.var(incohSubAmpsComplex)

In [ ]:
ampPath = "/nfs/freenas/tuph/e18/project/compass/analysis/tkumschier/ROOTPWA/test/TEST_FOLDER/ROOTPWA_MC_TEST0/amps/"
onlyfiles = [f for f in listdir(ampPath) if isfile(join(ampPath, f))]
nmbEvents = 500 # TODO: Automatisieren

print len(onlyfiles)
for files in range(0, 1): #len(onlyfiles)):
    print onlyfiles[files]
    ampFile = uproot.open(ampPath + onlyfiles[files])
    key = ampFile.keys()[0]
    
    incohSubAmpsReal = ampFile[key]['amplitude'].get('_incohSubAmps._real').array()
    incohSubAmpsImag = ampFile[key]['amplitude'].get('_incohSubAmps._imag').array()
    incohSubAmpsComplex = np.array(incohSubAmpsReal + 1j * incohSubAmpsImag)

    mean = sum(incohSubAmpsComplex * incohSubAmpsComplex.conjugate())/nmbEvents
    varReal = np.var(incohSubAmpsReal)
    varImag = np.var(incohSubAmpsImag)

    print len(incohSubAmpsReal), incohSubAmpsReal
    print mean[0].real, "+-", varReal
    print mean[0].imag, "+-", varImag
    print ""


In [ ]:
ampPath = "/nfs/freenas/tuph/e18/project/compass/analysis/tkumschier/ROOTPWA/test/ROOTPWA_MC_TEST/amps/"

onlyfiles = [f for f in listdir(ampPath) if isfile(join(ampPath, f))]
for i in range(0, len(onlyfiles)):
    print onlyfiles[i]

In [ ]:
onlyfiles = []
for filename in sorted(os.listdir(ampPath)):
    onlyfiles.append(filename)
    print filename

## Berechnung der Integralmatrix 

In [2]:
def prepareArray(origArray):
    preparedArray = np.zeros(len(origArray), dtype=complex)
    for i in range(0, len(origArray)):
        preparedArray[i] = origArray[i][0]
    return preparedArray

## Test Bereich

#ampPath = "/nfs/freenas/tuph/e18/project/compass/analysis/tkumschier/ROOTPWA/test/TEST_FOLDER/ROOTPWA_MC_TEST0/amps/"
ampPath = "/nfs/freenas/tuph/e18/project/compass/analysis/tkumschier/ROOTPWA/test/ROOTPWA_MC_TEST_500/amps/"

onlyfiles = []
for filename in sorted(os.listdir(ampPath)):
    onlyfiles.append(filename)
nmbWaves = len(onlyfiles)
ampFile1 = uproot.open(ampPath + onlyfiles[0])
nmbEvents = len(ampFile1[ampFile1.keys()[0]]['amplitude'].get('_incohSubAmps._real').array())
amp = np.zeros((nmbWaves, nmbWaves), dtype = complex)
sd = np.zeros((nmbWaves, nmbWaves), dtype = complex)
ampMatrix = np.zeros((nmbWaves,nmbEvents), dtype = complex)



for waveIndexI in range(0, nmbWaves):
    if ampMatrix[waveIndexI].all() == 0:
        ampFile1 = uproot.open(ampPath + onlyfiles[waveIndexI])
        key1 = ampFile1.keys()[0]

        incohSubAmpsReal1 = ampFile1[key1]['amplitude'].get('_incohSubAmps._real').array()
        incohSubAmpsImag1 = ampFile1[key1]['amplitude'].get('_incohSubAmps._imag').array()
        ampMatrix[waveIndexI] = prepareArray(np.array(incohSubAmpsReal1 + 1j * incohSubAmpsImag1))
        #print ampMatrix[waveIndexI]
        #raise ValueError('Exit.')


        
    
    for waveIndexJ in range(0, nmbWaves): #waveIndexI + 1): #nmbWaves):
        if ampMatrix[waveIndexJ].all() == 0:
            ampFile2 = uproot.open(ampPath + onlyfiles[waveIndexJ])
            key2 = ampFile2.keys()[0]

            incohSubAmpsReal2 = ampFile2[key2]['amplitude'].get('_incohSubAmps._real').array()
            incohSubAmpsImag2 = ampFile2[key2]['amplitude'].get('_incohSubAmps._imag').array()
            ampMatrix[waveIndexJ] = prepareArray(np.array(incohSubAmpsReal2 + 1j * incohSubAmpsImag2))

        integral = np.sum(ampMatrix[waveIndexI] * ampMatrix[waveIndexJ].conjugate())
        amp[waveIndexI][waveIndexJ] = amp[waveIndexJ][waveIndexI]
        
        sdReal = sum(pow((ampMatrix[waveIndexI] * ampMatrix[waveIndexJ].conjugate()).real, 2)).real
        sdImag = sum(pow((ampMatrix[waveIndexI] * ampMatrix[waveIndexJ].conjugate()).imag, 2)).real
        #print "Var1:", sdReal
        
        
        '''
        sdReal = 0.
        sdImag = 0.
        for subAmp in range(0, len(incohSubAmpsComplex1)):
            sdReal += pow((incohSubAmpsComplex1[subAmp] * incohSubAmpsComplex2.conjugate()[subAmp]).real, 2)
            sdImag += pow((incohSubAmpsComplex1[subAmp] * incohSubAmpsComplex2.conjugate()[subAmp]).imag, 2)
            #print (incohSubAmpsComplex1 * incohSubAmpsComplex2.conjugate()).real[subAmp][0].real, (incohSubAmpsComplex1[subAmp][0] * incohSubAmpsComplex2.conjugate()[subAmp][0]).real
        print "Var2:", sdReal
      '''
    

        sdReal = math.sqrt((sdReal - pow(amp[waveIndexI][waveIndexJ].real,2)/float(nmbEvents))/float(nmbEvents))
        sdImag = math.sqrt((sdImag - pow(amp[waveIndexI][waveIndexJ].imag,2)/float(nmbEvents))/float(nmbEvents))
        sdComplex = complex(sdReal, sdImag)        
        errorComplex = complex(sdReal/math.sqrt(nmbEvents), sdImag/math.sqrt(nmbEvents))
        
        sd[waveIndexI][waveIndexJ] = sdComplex
        sd[waveIndexJ][waveIndexI] = sdComplex

        amp[waveIndexI][waveIndexJ] = integral
        amp[waveIndexJ][waveIndexI] = integral      
        print "%i, %i: %s, %s, %s, %s" %(waveIndexI, waveIndexJ, amp[waveIndexI][waveIndexJ], amp[waveIndexI][waveIndexJ]/nmbEvents, sd[waveIndexI][waveIndexJ], errorComplex)
        #print np.cov(amp/nmbEvents)
            #raise ValueError('Exit.')
            
        if waveIndexJ == 10:
            print""
            break
    if waveIndexI == 5:
        break


In [30]:
#ampPath = "/nfs/freenas/tuph/e18/project/compass/analysis/tkumschier/ROOTPWA/test/TEST_FOLDER/ROOTPWA_MC_TEST0/amps/"
ampPath = "/nfs/freenas/tuph/e18/project/compass/analysis/tkumschier/ROOTPWA/test/ROOTPWA_MC_TEST_500/amps/"
ampPath = "/nfs/freenas/tuph/e18/project/compass/analysis/tkumschier/ROOTPWA/test/TEST_FOLDER_RUNS/ROOTPWA_MC_TEST0/amps/"


onlyfiles = []
for filename in sorted(os.listdir(ampPath)):
    onlyfiles.append(filename)
nmbWaves = len(onlyfiles)
ampFile1 = uproot.open(ampPath + onlyfiles[0])
nmbEvents = len(ampFile1[ampFile1.keys()[0]]['amplitude'].get('_incohSubAmps._real').array())
amp = np.zeros((nmbWaves, nmbWaves), dtype = complex)
sd = np.zeros((nmbWaves, nmbWaves), dtype = complex)
ampMatrix = np.zeros((nmbWaves,nmbEvents), dtype = complex)



for waveIndexI in range(0, nmbWaves):
    if ampMatrix[waveIndexI].all() == 0:
        ampFile1 = uproot.open(ampPath + onlyfiles[waveIndexI])
        key1 = ampFile1.keys()[0]

        incohSubAmpsReal1 = ampFile1[key1]['amplitude'].get('_incohSubAmps._real').array()
        incohSubAmpsImag1 = ampFile1[key1]['amplitude'].get('_incohSubAmps._imag').array()
        ampMatrix[waveIndexI] = prepareArray(np.array(incohSubAmpsReal1 + 1j * incohSubAmpsImag1))
        #print ampMatrix[waveIndexI]
        #raise ValueError('Exit.')


        
    
    for waveIndexJ in range(0, nmbWaves): #waveIndexI + 1): #nmbWaves):
        if ampMatrix[waveIndexJ].all() == 0:
            ampFile2 = uproot.open(ampPath + onlyfiles[waveIndexJ])
            key2 = ampFile2.keys()[0]

            incohSubAmpsReal2 = ampFile2[key2]['amplitude'].get('_incohSubAmps._real').array()
            incohSubAmpsImag2 = ampFile2[key2]['amplitude'].get('_incohSubAmps._imag').array()
            ampMatrix[waveIndexJ] = prepareArray(np.array(incohSubAmpsReal2 + 1j * incohSubAmpsImag2))

        integral = np.sum(ampMatrix[waveIndexI] * ampMatrix[waveIndexJ].conjugate())/nmbEvents
        amp[waveIndexI][waveIndexJ] = integral
        amp[waveIndexJ][waveIndexI] = integral     
        
        sdReal = sum(pow((ampMatrix[waveIndexI] * ampMatrix[waveIndexJ].conjugate()).real, 2)).real
        sdImag = sum(pow((ampMatrix[waveIndexI] * ampMatrix[waveIndexJ].conjugate()).imag, 2)).real
        #print "Var1:", sdReal
        
        
        '''
        sdReal = 0.
        sdImag = 0.
        for subAmp in range(0, len(incohSubAmpsComplex1)):
            sdReal += pow((incohSubAmpsComplex1[subAmp] * incohSubAmpsComplex2.conjugate()[subAmp]).real, 2)
            sdImag += pow((incohSubAmpsComplex1[subAmp] * incohSubAmpsComplex2.conjugate()[subAmp]).imag, 2)
            #print (incohSubAmpsComplex1 * incohSubAmpsComplex2.conjugate()).real[subAmp][0].real, (incohSubAmpsComplex1[subAmp][0] * incohSubAmpsComplex2.conjugate()[subAmp][0]).real
        print "Var2:", sdReal
      '''
    

        sdReal = math.sqrt(sdReal/float(nmbEvents) - pow(amp[waveIndexI][waveIndexJ].real,2))
        sdImag = math.sqrt(sdImag/float(nmbEvents) - pow(amp[waveIndexI][waveIndexJ].imag,2))
        sdComplex = complex(sdReal, sdImag)        
        errorComplex = complex(sdReal/math.sqrt(nmbEvents), sdImag/math.sqrt(nmbEvents))
        
        sd[waveIndexI][waveIndexJ] = sdComplex
        sd[waveIndexJ][waveIndexI] = sdComplex
 
        #print "%i, %i: %s, %s, %s" %(waveIndexI, waveIndexJ, amp[waveIndexI][waveIndexJ], sd[waveIndexI][waveIndexJ], errorComplex)
        print "%i, %i: %s, %s" %(waveIndexI, waveIndexJ, amp[waveIndexI][waveIndexJ], errorComplex)
        #print np.cov(amp/nmbEvents)
            #raise ValueError('Exit.')
            
        if waveIndexJ == 10:
            print""
            break
    if waveIndexI == 5:
        break


0, 0: (0.45556524307684654+0j), (0.0361104138644+0j)
0, 1: (1.1508793683788363+0.2653219786795264j), (0.0896518721399+0.0241797862301j)
0, 2: (0.007776633284392142+0.05581450507022344j), (0.00561797466924+0.00502596777356j)
0, 3: (0.21787691120685787+0.0023589255052255433j), (0.0172977625131+0.000385756494223j)
0, 4: (-0.094812296879-0.17031239527163222j), (0.0699422374677+0.102795500257j)
0, 5: (0.3947859671144579+0.08074035633773151j), (0.0259076765678+0.0261573942337j)
0, 6: (-0.01075710899954522+0.052912384113909035j), (0.00787456558088+0.0133680676482j)
0, 7: (0.06584314787366563-0.0259726683124114j), (0.040242964667+0.0166529597794j)
0, 8: (0.1715273615538611-0.08897264925707209j), (0.0976919464526+0.0512191237159j)
0, 9: (0.03100495821032507-0.011822227453603708j), (0.0191161729561+0.0079303804476j)
0, 10: (-0.010491687283226728-0.032317025340841944j), (0.0280639191293+0.0370984111341j)

1, 0: (1.1508793683788363-0.2653219786795264j), (0.0896518721399+0.0241797862301j)
1, 1: (3.

In [ ]:
a = np.empty(5, dtype=complex)
b = np.empty(5, dtype=complex)
for i in range(0, 5):
    a[i] = complex(i+1, 2 * (i+1))
    b[i] = complex(100-i, i*10)
    

print a * b.conjugate()
print np.sum(a*b.conjugate())

val = 0
for i in range(0, 5):
    val += a[i] * b[i].conjugate()
print val

In [ ]:
arr1 = np.array([1+2j, 5+10j])
print arr1
print np.square(arr1)
print arr1.real

print ""
arr2 = np.array([[1+2j], [5+10j]])
print arr2
print np.square(arr2)
print arr2.real

In [ ]:
print np.empty((2,3))
print np.empty((2,3))[0]